# 0.0. IMPORTS

In [ ]:
import math
import numpy as np
import pandas as pd
import inflection

import seaborn as sns

from matplotlib import pyplot as plt

import inflection
from IPython.core.display import HTML
from IPython.display import Image
import datetime

from scipy import stats as ss

## 0.1. Helper Functions

## 0.2. Loading Data

In [ ]:
df_sales_raw = pd.read_csv('data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('data/store.csv', low_memory=False)

# merge
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')

In [ ]:
df_raw.sample(5)

# 1.0. DATA DESCRIPTION

In [ ]:
# copy original data
df1 = df_raw.copy()

## 1.1. Rename Columns

In [ ]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
           'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
           'CompetitionDistance', 'CompetitionOpenSinceMonth',
           'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
           'Promo2SinceYear', 'PromoInterval']

# Apply snakecase to columns names
snakecase = lambda x: inflection.underscore( x )
cols_new = list(map(snakecase, cols_old))

#rename
df1.columns = cols_new

## 1.2. Data Dimensions

In [ ]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Cols: {}'.format(df1.shape[1]))

## 1.3. Data Types

In [ ]:
df1['date'] = pd.to_datetime( df1['date'] )
df1.dtypes

## 1.4. Check NA

In [ ]:
df1.isna().sum()

## 1.5. Fillout NA

In [ ]:
df1.sample()

In [ ]:
df1['competition_distance'].max()

In [ ]:
#competition_distance
# market analysis assumption: position without values means a far competition distance, higher than the
# maximum value from the raw data.
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)

# competition_open_since_month
# Assumption: for NA values apply the value from "date" column, because the competition does exist and it can affect
# over time the sales performance.
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

#competition_open_since_year
# Same assumption as competition_open_since_month
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

#promo2_since_week
# Assumptions: 1. NA values mean the store did not take place in promo2. 2. apply the value from "date" column.
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

#promo2_since_year
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

#promo_interval
# Dictionary for Months
month_map = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

# Fill NA values with zero.
df1['promo_interval'].fillna(0, inplace=True)

# Add new column with month information
df1['month_map'] = df1['date'].dt.month.map(month_map)

# New column that shows if at that date the store was in promo2.
df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

In [ ]:
df1.isna().sum()

## 1.6. Change Types

In [ ]:
df1.dtypes

In [ ]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)

df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)


In [ ]:
df1.dtypes

## 1.7. Descriptive Statistical

In [ ]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])


### 1.7.1 Numerical Attributes

In [ ]:
#Central Tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max()-x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# concatenate
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

In [ ]:
sns.distplot(df1['competition_distance'])
sns.displot(df1['competition_distance'])
sns.histplot(df1['competition_distance'])

### 1.7.2 Categorical Attributes

In [ ]:
cat_attributes.apply(lambda x: x.unique().shape[0])

In [ ]:
aux1 = df1[(df1['state_holiday'] != '0') &  (df1['sales'] > 0)]
# plt.subplot(1, 3, 1)
sns.boxplot(x='state_holiday', y='sales', data=aux1)



In [ ]:
# plt.subplot(1, 3, 2)
sns.boxplot(x='store_type', y='sales', data=aux1)


In [ ]:

# plt.subplot(1, 3, 3)
sns.boxplot(x='assortment', y='sales', data=aux1)

# 2.0. Feature Engineering

In [ ]:

Image('img/Mindmaphypothesis.png')

## 2.1. Criacao de Hipteses

## 2.1.1 Hipoteses Loja

**1.** Lojas com maior quadro de funcionarios deveriam vender mais.

**2.** Lojas com maior estoque deveriam vender mais.

**3.** Lojas com maior porte deveriam vender mais.

**4.** Lojas com maior sortimento deveriam vender mais.

**5.** Lojas com competidores mais proximos deveriam vender menos.

**6.** Lojas com competidores à mais tempo deveriam vender mais.

## 2.1.2 Hipoteses Produto

**1.** Lojas que investem mais em Marketion deveriam vender mais.

**2.** Lojas que expoem mais o produto deveriam vender mais.

**3.** Lojas com precos menores deveriam vender mais.

**4.** Lojas com precos menores por mais tempo deveriam vender mais.

**5.** Lojas com promocoes agressivas(descontos maiores) deveriam vender mais.

**6.** Lojas com mais dias de promocao deveriam vender mais.

**7.** Lojas com mais promocoes consecutivas deveriam vender mais.


## 2.1.3 Hipoteses Tempo

**1.** Lojas abertaas durante o feriado de Natal deveriam vender mais.

**2.** Lojas deveriam vender mais ao longo dos anos (Store age ).

**3.** Lojas deveriam vender mais no segundo semestre do ano.

**4.** Lojas deveriam vender mais depois do dia 10 de cada mes.

**5.** Lojas deveriam vender menos aos fins de semana.

**6.** Lojas deveriam vender menos durante os feriados escolares.

## 2.2. Lista Final de Hipoteses

**1.** Lojas com maior sortimento deveriam vender mais.

**2.** Lojas com competidores mais proximos deveriam vender menos.

**3.** Lojas com competidores à mais tempo deveriam vender mais.

**4.** Lojas com promocoes agressivas(descontos maiores) deveriam vender mais.

**5.** Lojas com mais dias de promocao deveriam vender mais.

**6.** Lojas com mais promocoes consecutivas deveriam vender mais.

**7.** Lojas abertaas durante o feriado de Natal deveriam vender mais.

**8.** Lojas deveriam vender mais ao longo dos anos (Store age ).

**9.** Lojas deveriam vender mais no segundo semestre do ano.

**10.** Lojas deveriam vender mais depois do dia 10 de cada mes.

**11.** Lojas deveriam vender menos aos fins de semana.

**12.** Lojas deveriam vender menos durante os feriados escolares.


In [ ]:
df2 = df1.copy()

## 2.3. Feature Engineering

In [ ]:
# year
df2['year'] = df2['date'].dt.year
# month
df2['month'] = df2['date'].dt.month
# day
df2['day'] = df2['date'].dt.day
# week of the year
# df2['week_of_year'] = df2['date'].dt.weekofyear
df2['week_of_year'] = df2['date'].dt.isocalendar().week
# year week
df2['year_week'] = df2['date'].dt.strftime('%Y-%W')

# competition since
df2['competition_since'] = df2.apply( lambda x: datetime.datetime(year=x['competition_open_since_year'], month=x['competition_open_since_month'], day=1 ), axis=1 )
df2['competition_time_month']= ((df2['date'] - df2['competition_since'])/30).apply(lambda x: x.days).astype(int)

# promo since
df2['promo_since'] = df2['promo2_since_year'].astype(str)+'-'+df2['promo2_since_week'].astype(str)
df2['promo_since'] = df2['promo_since'].apply( lambda x: datetime.datetime.strptime( x + '-1', '%Y-%W-%w' ) - datetime.timedelta(days=7))
df2['promo_time_week'] = ((df2['date'] - df2['promo_since'])/ 7).apply(lambda x: x.days).astype(int)
                                              
# assortment
df2['assortment'] = df2['assortment'].apply(lambda x : 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended' )
# stateholiday
df2['state_holiday'] = df2['state_holiday'].apply(lambda x : 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day')



In [ ]:
df2.sample(20).T

# 3.0. Passo 03 - Filtragem de Variaveis

In [ ]:
df3 = df2.copy()
df3.columns

In [ ]:
df3.sample(5)

## 3.1. Filtragem das linhas 

In [ ]:
df3 = df3[(df3['open'] != 0) & (df3['sales'] > 0)]


## 3.2. Selecao das Colunas 

In [ ]:
cols_drop = ['customers', 'open', 'promo_interval', 'month_map']
df3 = df3.drop(cols_drop, axis = 1)

In [ ]:
df3.sample(5)

In [ ]:
df3.columns

# 4.0. Passo 04 - Analise exploratoria dos dados


In [ ]:
df4 = df3.copy()

In [ ]:
df4.sample(5)

## 4.1. Analise UniVariada

### 4.1.1 Response Variable

In [ ]:
sns.distplot(df4['sales']);

In [ ]:
sns.histplot(df4['sales'], element="step", stat="density", kde=True );

### 4.1.2 Numerical Variable

In [ ]:
num_attributes.hist(bins=25,  figsize=(24,20));

### 4.1.3 Categorical Variable

In [ ]:
df4['assortment'].drop_duplicates()

In [ ]:
sns.color_palette()

# state_holiday	
plt.subplots(figsize=(18,22 ))
plt.subplot(3, 2, 1 )
a= df4 [df4['state_holiday'] != 'regular_day']
sns.countplot(a['state_holiday'])

# sns.countplot(df4['state_holiday'])

plt.subplot(3, 2, 2)
sns.kdeplot(df4[df4['state_holiday'] == 'public_holiday']['sales'], label='public_holiday', multiple="stack")
sns.kdeplot(df4[df4['state_holiday'] == 'easter_holiday']['sales'], label='easter_holiday', multiple="stack")
sns.kdeplot(df4[df4['state_holiday'] == 'christmas']['sales'], label='christmas', multiple="stack")

# store_type

plt.subplot(3, 2, 3)
sns.countplot(df4['store_type'])



plt.subplot(3, 2, 4)
sns.kdeplot(df4[df4['store_type'] == 'a']['sales'], label='a', multiple="stack")
sns.kdeplot(df4[df4['store_type'] == 'b']['sales'], label='b', multiple="stack")
sns.kdeplot(df4[df4['store_type'] == 'c']['sales'], label='c', multiple="stack")
sns.kdeplot(df4[df4['store_type'] == 'd']['sales'], label='d', multiple="stack")

# assortment

plt.subplot(3, 2, 5)
sns.countplot(df4['assortment'])


plt.subplot(3, 2, 6)
sns.kdeplot(df4[df4['assortment'] == 'basic']['sales'], label='basic', multiple="stack")
sns.kdeplot(df4[df4['assortment'] == 'extended']['sales'], label='extended', multiple="stack")
sns.kdeplot(df4[df4['assortment'] == 'extra']['sales'], label='extra', multiple="stack")



## 4.2. Analise Bivariada

### H1. Lojas com maiores sortimentos deveriam vender mais.

In [ ]:
df4.columns

In [ ]:
plt.subplots(figsize=(18,20 ))
aux1 = df4[['assortment', 'sales']].groupby('assortment').sum().reset_index()
sns.barplot(x= 'assortment', y= 'sales', data=aux1);
# o extra seria os maiores assortiments

aux2 = df4[['year_week', 'assortment', 'sales']].groupby(['year_week', 'assortment']).sum().reset_index()
aux2.pivot(index='year_week', columns='assortment', values='sales').plot()

aux3 = aux2[aux2['assortment'] == 'extra']
aux3.pivot(index='year_week', columns='assortment', values='sales').plot()

In [ ]:
aux3.head()

### H2. Lojas com cvompetidores mais proximos deveriam vender menos.

In [ ]:
plt.subplots(figsize=(20,8 ))
aux1 = df4[['competition_distance', 'sales']].groupby('competition_distance').sum().reset_index()

plt.subplot(1, 3, 1)
sns.scatterplot( x='competition_distance', y='sales', data=aux1)


plt.subplot(1, 3, 2)
bins = list( np.arange(0, 20000, 1000))
aux1['competition_distance_binned'] = pd.cut( aux1['competition_distance'], bins=bins)
aux2 = aux1[['competition_distance_binned', 'sales']].groupby('competition_distance_binned').sum().reset_index();
sns.barplot(x= 'competition_distance_binned', y= 'sales', data=aux2);


plt.subplot(1, 3, 3)
# aux3 = aux1[aux1['competition_distance'].astype(int)]
# correlation = aux3.corr(method='pearson')
# sns.heatmap(aux3.corr( method='pearson'), annot=True);



In [ ]:
# TypeError: float() argument must be a string or a number, not 'pandas._libs.interval.Interval'
# competition_distance            float64
# sales                             int64
# competition_distance_binned    category
# dtype: object

aux3.head(10)

### H3. Lojas com competidores à mais tempo deveriam vender mais.

In [ ]:
df4.columns

In [ ]:
aux1 = df4[['competition_time_month', 'sales']].groupby('competition_time_month').sum().reset_index()
aux2 = aux1[(aux1['competition_time_month']  < 120)  & (aux1['competition_time_month']  != 0)]


plt.subplots(figsize=(20,16 ))
plt.subplot(1, 2, 1)
sns.barplot(x='competition_time_month', y='sales', data=aux2 );
plt.xticks(rotation=90);

plt.subplot(1, 2, 2)
sns.regplot(x='competition_time_month', y='sales', data=aux2 );

In [ ]:
aux1.head()

## 4.3. Analise MultiVariada

### 4.3.1 Numerical Attributes

In [ ]:

plt.subplots(figsize=(20,13))

correlation = num_attributes.corr(method='pearson')
sns.heatmap(correlation, annot=True);

### 4.3.2 Categorical Attributes

In [ ]:
a = df4.select_dtypes(include='object')
a.head()
# cat_attributes.head()

In [ ]:
cm = pd.crosstab(a['state_holiday'], a['store_type'])

In [ ]:
cm

In [ ]:
cm2 = a[['state_holiday', 'store_type']].values

In [ ]:
cm2

In [ ]:
chi2 = ss.chi2_contingency( cm )[0]

n = cm2.sum()

r, k = cm2.shape

In [ ]:
print(chi2)


In [ ]:
print(n)


In [ ]:
print(r)


In [ ]:
print(k)

In [ ]:
v = np.sqrt((chi2/n)/(min(k-1,r-1)))

In [ ]:
v

# 5.0. Passo 05 - Data Preparation

In [ ]:
df5 = df4.copy()

## 5.1. Normalizacao

## 5.2. Rescaling

## 5.3. Transformation